# IMPORTS

In [ ]:
import pandas as pd

# import json
# from collections import defaultdict

import my_pickle as mp
import my_ect_split as ms
import my_dataframes as md

from importlib import reload

In [ ]:
reload(md)

# READ IN

In [ ]:
# read in message data
message_df = md.get_messages()
convo_df = md.get_conversations(message_df)
last_df = md.get_lastmessage()

# ECT SPLIT

In [ ]:
# find the split!
cutoff = ms.ect_find_split(convo_df, .7)
pd.to_datetime(cutoff*1000000)

In [ ]:
# do the split!
safe_convo_df = ms.ect_make_split(convo_df, cutoff)
pd.to_datetime(safe_convo_df.timestamp.max()*1000000)

# COMBINE USER AND MESSAGE FILES

NB: Have to do after ECT split!

In [ ]:
responsiveness_df = md.get_responsiveness(convo_df)

In [ ]:
# find average attractiveness
average_attractiveness = responsiveness_df[responsiveness_df.attractiveness >0].mean()['attractiveness']
average_selectivity = responsiveness_df[responsiveness_df.selectivity >0].mean()['selectivity']

In [ ]:
# Combine with user table
user_df = md.get_user_data()
user_df = user_df.join(responsiveness_df)
user_df = user_df.fillna({'messages_sent' : 0,'responses_received': 0,'attractiveness':average_attractiveness,
                          'messages_received': 0,'responses_sent': 0, 'selectivity':average_selectivity})
user_df.head()
# issue where 0 and 0 can be 0 or average b/c of way joined. ignore for now.

In [ ]:
print(len(user_df))
print(user_df.onboarded.sum())

# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [ ]:
mp.pickle_it(user_df,'user_df')
# mp.pickle_it(message_df,'message_df')  NOT ECT SAFE!
mp.pickle_it(safe_convo_df,'convo_df')  # make sure pickling safe stuff
# pickle_it(lastmessage_df,'lastmessage_df') NOT ECT SAFE!